# Lab 4: PyTorch Fundamentals - SOLUTIONS

**Day 2 - Deep Learning**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

torch.manual_seed(42)
np.random.seed(42)

## Exercise 1: Tensor Basics - SOLUTION

In [ ]:
def create_tensors():
    tensor_from_list = torch.tensor([1, 2, 3, 4])
    zeros = torch.zeros(3, 3)
    ones = torch.ones(3, 3)
    random_tensor = torch.rand(3, 3)
    from_numpy = torch.from_numpy(np.array([[1, 2], [3, 4]]))
    return tensor_from_list, zeros, ones, random_tensor, from_numpy

def tensor_operations():
    a = torch.tensor([[1, 2], [3, 4]], dtype=torch.float32)
    b = torch.tensor([[5, 6], [7, 8]], dtype=torch.float32)
    return a + b, a * b, a @ b, torch.sum(a), torch.mean(a)

# Test
for i, t in enumerate(create_tensors()):
    print(f"Tensor {i+1}: shape={t.shape}")
print("\nOperations:", tensor_operations())

## Exercise 2: Automatic Differentiation - SOLUTION

In [ ]:
def compute_gradients():
    x = torch.tensor([2.0], requires_grad=True)
    y = x**2 + 3*x + 1
    y.backward()
    return x.grad.item()

def chain_rule_example():
    x = torch.tensor([3.0], requires_grad=True)
    g = 2 * x
    y = g ** 2
    y.backward()
    return x.grad.item()

print(f"Gradient of x^2 + 3x + 1 at x=2: {compute_gradients()} (expected: 7)")
print(f"Gradient of (2x)^2 at x=3: {chain_rule_example()} (expected: 24)")

## Exercise 3: Neural Networks - SOLUTION

In [ ]:
class SimpleNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 4)
        self.layer2 = nn.Linear(4, 1)
        self.activation = nn.ReLU()
    
    def forward(self, x):
        x = self.activation(self.layer1(x))
        return self.layer2(x)

class FlexibleNet(nn.Module):
    def __init__(self, layer_sizes):
        super().__init__()
        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i+1]))
    
    def forward(self, x):
        for i, layer in enumerate(self.layers):
            x = layer(x)
            if i < len(self.layers) - 1:
                x = torch.relu(x)
        return x

# Test
simple_net = SimpleNet()
print(simple_net)
print(f"Parameters: {sum(p.numel() for p in simple_net.parameters())}")

## Exercise 4: Training Loop - SOLUTION

In [ ]:
X_xor = torch.tensor([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=torch.float32)
y_xor = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

class XORNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(2, 4)
        self.layer2 = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.relu(self.layer1(x))
        x = self.sigmoid(self.layer2(x))
        return x

def train_xor_network(model, X, y, epochs=1000, lr=0.1):
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    losses = []
    
    for epoch in range(epochs):
        predictions = model(X)
        loss = criterion(predictions, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
    
    return losses

# Train
xor_model = XORNet()
losses = train_xor_network(xor_model, X_xor, y_xor, epochs=2000, lr=0.5)

print("After training:")
with torch.no_grad():
    for x, actual in zip(X_xor, y_xor):
        pred = xor_model(x.unsqueeze(0))
        print(f"  {x.tolist()} -> {pred.item():.4f} (expected {actual.item()})")

plt.plot(losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('XOR Training')
plt.show()

## Exercise 5: nn.Sequential - SOLUTION

In [ ]:
def create_sequential_network():
    return nn.Sequential(
        nn.Linear(10, 64),
        nn.ReLU(),
        nn.Linear(64, 32),
        nn.ReLU(),
        nn.Linear(32, 1),
        nn.Sigmoid()
    )

seq_model = create_sequential_network()
print(seq_model)
print(f"Parameters: {sum(p.numel() for p in seq_model.parameters())}")

## Exercise 6: Complete Pipeline - SOLUTION

In [ ]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=500, noise=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32).reshape(-1, 1)
X_test_t = torch.tensor(X_test, dtype=torch.float32)
y_test_t = torch.tensor(y_test, dtype=torch.float32).reshape(-1, 1)

def train_classifier(X_train, y_train, X_test, y_test, epochs=200):
    model = nn.Sequential(
        nn.Linear(2, 16),
        nn.ReLU(),
        nn.Linear(16, 8),
        nn.ReLU(),
        nn.Linear(8, 1),
        nn.Sigmoid()
    )
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    
    train_losses = []
    test_accuracies = []
    
    for epoch in range(epochs):
        predictions = model(X_train)
        loss = criterion(predictions, y_train)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_losses.append(loss.item())
        
        if epoch % 10 == 0:
            with torch.no_grad():
                test_pred = (model(X_test) > 0.5).float()
                acc = (test_pred == y_test).float().mean().item()
                test_accuracies.append(acc)
    
    return model, train_losses, test_accuracies

model, losses, accs = train_classifier(X_train_t, y_train_t, X_test_t, y_test_t, epochs=500)

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(losses)
axes[0].set_title('Training Loss')
axes[1].plot(range(0, 500, 10), accs)
axes[1].set_title('Test Accuracy')
plt.show()

print(f"Final accuracy: {accs[-1]:.2%}")

## Checkpoint

Lab 4 complete! **Next:** Lab 5 - NLP Basics